In [1]:
import pandas as pd
import numpy as np

'''
Author: Enrico Ceccolini
    Drop the jobs with zero timepoints.
    Split between trainset and testset
'''

datadir = "/datasets/eurora_data/db1/"

#infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow" # there are also short jobs

#nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

# clean user names
jobs['user'] = jobs.apply(lambda row: row['user'].split('@')[0], axis=1)


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 336787 long jobs in the Alina's interval


In [3]:
jobs.columns


Index(['job_id_string', 'job_name', 'queue', 'start_time', 'run_start_time',
       'end_time', 'user', 'node_req', 'cpu_req', 'mem_req', 'time_req',
       'deleted', 'dependency', 'exit_status', 'pbs_exit_code', 'gpu_req',
       'mic_req', 'real_pow', 'ran_alone', 'real_pow_quality', 'n_2_1',
       'n_2_2', 'n_3_1', 'job_tot_timepoints', 'job_timepoints', 'good_nodes'],
      dtype='object')

In [4]:
# no jobs with real_pow smaller than 0
print(jobs[jobs['real_pow'] <= 0]['real_pow'].shape[0])
print(jobs[jobs['real_pow'] == 0]['real_pow'].shape[0])


8527
8527


fill the nan time_req using wall times

In [5]:
jobs['time_req'].unique()

array(['04:00', '01:00', '06:00', '08:00', '03:00', '00:20', '00:30',
       '02:00', '03:30', '00:10', '00:02', '00:00', '03:59', '01:30',
       '00:29', '00:09', '00:15', '00:05', '00:04', '00:01', '03:29',
       '02:30', '00:08', '07:59', '01:10', '00:03', '00:35', '01:20',
       '00:40', '--', '03:20', '01:29', nan], dtype=object)

In [6]:
# debug
jobs[jobs['time_req'].isnull()]['queue'].unique()

array(['parallel', 'debug', 'system', 'route', 'np_longpar', 'p_devel',
       'meteopar', 'R588271', 'R588272', 'R588273', 'visual', 'R588274',
       'R588275', 'R588276', 'R588277', 'R588278', 'R588279', 'R588280',
       'R588281', 'R588282', 'R588283', 'R588642', 'R588286', 'R588287',
       'R588288', 'R588289', 'R588290', 'R588291', 'R588293', 'R588294',
       'R588295', 'R588296', 'R588297', 'R588298', 'R628233', 'R629002',
       'R629001', 'R628999', 'R629000', 'R628996', 'R628994', 'R628993',
       'R628990', 'R628989', 'R635731', 'R769563', 'R771865', 'R769564',
       'R959715', 'R959716', 'R959717', 'R959718', 'R959719', 'R959720',
       'R959721', 'R959722', 'R1010998', 'R1011000', 'R1013900',
       'R1013903', 'R1013901', 'R1013904', 'R1013902', 'R1010870',
       'R1172721', 'R1172722', 'R1172723', 'R1172724', 'R1172725',
       'R1208877', 'R1201947', 'R1201948', 'R1201949', 'R1201950',
       'R1201951', 'R1201952', 'R1287258', 'R1287259', 'R1287260',
       'R12

In [7]:
jobs.shape[0]

336787

In [8]:
jobs.loc[jobs['time_req'] == '--', 'time_req'] = np.NaN
print(jobs[jobs['time_req'].isnull()].shape[0])

277130


In [9]:
#debug wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'debug'), 'time_req' ] = '00:30'
print(jobs[jobs['time_req'].isnull()].shape[0])

229501


In [10]:
#p_devel wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'p_devel'), 'time_req' ] = '01:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

229386


In [11]:
#parallel wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'parallel'), 'time_req' ] = '04:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

65106


In [12]:
#np_longpar wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'np_longpar'), 'time_req' ] = '08:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

62842


In [13]:
#all the others wall time
jobs.loc[ jobs['time_req'].isnull(), 'time_req'] = '04:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

0


In [14]:
jobs[jobs['real_pow_quality'] <= 0]

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
17,498474.node129,WW_SWAN,R472836,2014-03-31 05:19:41,2014-03-31 05:19:41,2014-03-31 05:27:46,a07smr01,1,1,2,...,NaN,9.687526,True,0.0,0,0,1,97,0,1
23,498480.node129,SWAN_SNE,R472836,2014-03-31 07:40:10,2014-03-31 07:40:17,2014-03-31 07:54:40,a07smr01,2,32,28,...,NaN,736.987717,True,0.0,0,0,2,344,0,2
66,498541.node129,marco.sh,parallel,2014-03-31 10:16:05,2014-03-31 10:16:05,2014-03-31 10:31:10,ebodo000,1,4,4,...,NaN,369.740816,True,0.0,0,0,1,181,0,1
68,498543.node129,abq_parallel,parallel,2014-03-31 10:19:26,2014-03-31 10:19:26,2014-03-31 10:20:59,dcrivell,1,8,5,...,NaN,131.277029,True,0.0,0,0,1,18,0,1
80,498556.node129,invert_test.sh,debug,2014-03-31 10:52:49,2014-03-31 10:52:49,2014-03-31 10:53:00,mschroec,1,1,5,...,NaN,0.000000,True,0.0,0,0,1,2,0,1
94,498570.node129,invert_test.sh,debug,2014-03-31 11:03:19,2014-03-31 11:03:19,2014-03-31 11:03:25,mschroec,1,1,5,...,NaN,0.000000,True,0.0,0,0,1,1,0,1
110,498629.node129,h2ogeo_opt,parallel,2014-03-31 11:13:24,2014-03-31 11:54:23,2014-03-31 11:54:37,mpividor,4,64,4,...,NaN,880.188007,True,0.0,0,0,4,8,0,4
112,498631.node129,invert_test.sh,debug,2014-03-31 11:14:14,2014-03-31 11:14:14,2014-03-31 11:14:19,mschroec,1,1,5,...,NaN,0.000000,True,0.0,0,0,1,1,0,1
155,498674.node129,Halo81_MP2_VTZ_,debug,2014-03-31 11:15:15,2014-03-31 11:15:16,2014-03-31 11:15:21,apietro1,1,1,4,...,NaN,2.001383,True,0.0,0,0,1,1,0,1


In [15]:
# ['real pow'] < 0: when we calculate the real job pow approximation we may have some negative results when 
# we subtract the mean idle pow of the node
# ['real_pow_quality'] < 0: jobs with 0 timepoints has 0 real_pow_quality. We eant jobs with at least one timepoint
print(jobs.shape[0])
jobs_q1 = jobs[jobs['real_pow_quality'] > 0]
jobs_q1 = jobs_q1[jobs_q1['real_pow'] > 0] 
print(jobs_q1.shape[0])

336787
165126


In [16]:
# with jobs['real_pow_quality'] > 0
#165126

# with jobs['real_pow_quality'] >= 0
#311172

In [17]:
print("{} distinct users".format(jobs_q1['user'].unique().shape[0]))

265 distinct users


In [18]:
jobs_q1.groupby(['user', 'job_name']).size().shape[0]

12301

In [19]:
jobs_train = jobs_q1[pd.to_datetime(jobs_q1['end_time']) <= train_end_time]
jobs_train = jobs_train[pd.to_datetime(jobs_train['run_start_time']) >= train_start_time]

print("train set contains {} records".format(jobs_train.shape[0]))

jobs_test = jobs_q1[pd.to_datetime(jobs_q1['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) >= train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 146901 records
test set contains 18224 records


In [20]:
print("{} unique jobs in the trainset".format(jobs_train.groupby(['user', 'job_name']).size().shape[0]))
print("{} unique jobs in the testset".format(jobs_test.groupby(['user', 'job_name']).size().shape[0]))


6491 unique jobs in the trainset
6675 unique jobs in the testset


In [21]:
outfile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset"
jobs_train.to_csv(outfile_jobs_trainset + ".csv")

outfile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset"
jobs_test.to_csv(outfile_jobs_testset + ".csv")